In [ ]:
import os
os.chdir('/Users/andrei-macpro/Documents/Data/Audio/overlapping_speech/no_os_meal')
import torch
import numpy as np

In [ ]:
sad = torch.hub.load('pyannote/pyannote-audio', 'sad_ami')
scd = torch.hub.load('pyannote/pyannote-audio', 'scd_ami')
emb = torch.hub.load('pyannote/pyannote-audio', 'emb_ami')

In [ ]:
test_file = {'audio': '1043_meal.wav'}

In [ ]:
sad_scores = sad(test_file)

In [ ]:
from pyannote.audio.utils.signal import Binarize
binarize = Binarize(offset=0.52, onset=0.52, log_scale=True, 
                    min_duration_off=0.1, min_duration_on=0.1)

# speech regions (as `pyannote.core.Timeline` instance)
speech = binarize.apply(sad_scores, dimension=1)


In [ ]:
scd_scores = scd(test_file)

# detect peaks and return speaker homogeneous segments 
# NOTE: both alpha/min_duration values were tuned on AMI dataset.
# you might need to use different values for better results.
from pyannote.audio.utils.signal import Peak
peak = Peak(alpha=0.10, min_duration=0.10, log_scale=True)

# speaker change point (as `pyannote.core.Timeline` instance)
partition = peak.apply(scd_scores, dimension=1)

In [ ]:
speech_turns = partition.crop(speech)

In [ ]:
embeddings = emb(test_file)

In [ ]:

chunks = embeddings.sliding_window
print(f'Embeddings were extracted every {1000 * chunks.step:g}ms on {1000 * chunks.duration:g}ms-long windows.')

In [ ]:
from pyannote.core import Timeline
long_turns = Timeline(segments=[s for s in speech_turns if s.duration > 1.])

In [ ]:
X, Y = [], []
for segment in long_turns:
    # "strict" only keeps embedding strictly included in segment
    x = embeddings.crop(segment, mode='strict')
    # average speech turn embedding
    X.append(np.mean(x, axis=0))

    # keep track of speaker label (for later scatter plot)
    y = test_file['annotation'].argmax(segment)
    Y.append(y)

X = np.vstack(X)
_, y_true = np.unique(Y, return_inverse=True)

In [ ]:
X, Y = [], []
for segment in long_turns:
    # "strict" only keeps embedding strictly included in segment
    x = embeddings.crop(segment, mode='strict')
    # average speech turn embedding
    X.append(np.mean(x, axis=0))


In [ ]:
len(X)

In [ ]:
X

In [ ]:
len(X)